In [708]:
import datetime
import pandas as pd
import numpy as np
from sklearn import linear_model
from google.cloud import bigquery
#import datalab.storage as gcs #v1.1.5
#import datalab.storage as gcs #v1.1.5
from datetime import timedelta

In [709]:
end_date = "2020-06-04"

In [710]:
%%bigquery df_0
SELECT
  leg_true_region_pair_nondirectional as region_pair
FROM
  `skytra-benchmark-rnd.iata.R1` WHERE dt_of_issue > "2018-10-10"
GROUP BY 
leg_true_region_pair_nondirectional

 


In [711]:
regions = df_0.region_pair.unique()
skytra_regions = regions
regions

array(['Asia/Pacific-CIS', 'Africa-Latin America',
       'Asia/Pacific-Latin America', 'Middle East-Middle East',
       'Europe-Europe', 'CIS-CIS', 'Africa-Africa',
       'Asia/Pacific-North America', 'Europe-Latin America', 'Africa-CIS',
       'Asia/Pacific-Middle East', 'CIS-Middle East',
       'Latin America-Middle East', 'CIS-North America',
       'CIS-Latin America', 'Asia/Pacific-Europe',
       'Latin America-Latin America', 'Asia/Pacific-Asia/Pacific', None,
       'Middle East-North America', 'Africa-Middle East', 'CIS-Europe',
       'Latin America-North America', 'Europe-Middle East',
       'North America-North America', 'Africa-Europe',
       'Europe-North America', 'Africa-North America',
       'Africa-Asia/Pacific'], dtype=object)

# Creating yield indices

In [712]:
%%bigquery df_0
SELECT
  region_pair,
  flight_op_carrier,
  dt_of_issue,
  SUM(leg_revenue)/(SUM(dist_km)) AS yield
FROM
  `skytra-benchmark-rnd.matching.R6`
WHERE
  transaction_code = "I"  
GROUP BY
  region_pair,
  flight_op_carrier,
  dt_of_issue

In [649]:
# The main IATA index dataframe
skytra_sf_df= df_0
skytra_sf_df.tail()




,region_pair,flight_op_carrier,dt_of_issue,yield
3561650,"Asia/Pacific-Latin America,Asia/Pacific-North ...",G3,2019-07-09,0.133291407
3561651,"Asia/Pacific-Latin America,Asia/Pacific-North ...",QR,2019-09-27,0.121454233
3561652,"Asia/Pacific-Latin America,Asia/Pacific-North ...",LX,2018-03-05,0.142942337
3561653,"Asia/Pacific-Latin America,Asia/Pacific-North ...",CI,2018-09-28,0.143102249
3561654,"Asia/Pacific-Latin America,Asia/Pacific-North ...",OZ,2019-05-31,0.124327181


In [650]:
skytra_sf_df = skytra_sf_df[skytra_sf_df["dt_of_issue"]  < pd.to_datetime(end_date)]

In [651]:
skytra_sf_df["region_pair"] = np.where(skytra_sf_df["region_pair"].isnull(),
                                       "multiple",
                                       skytra_sf_df["region_pair"])




In [652]:
region = regions[0]
tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
tmp_table["yield"] = tmp_table["yield"].astype('float')
index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='yield', columns='flight_op_carrier')
#index_tmp = index_tmp.fillna(method='bfill')
#index_tmp = index_tmp.fillna(0)
#index_tmp = index_tmp.rolling(28).mean()
#region

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [713]:
airline_index = pd.DataFrame()

for region in regions:
    try:
            tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
            tmp_table["yield"] = tmp_table["yield"].astype('float')
            index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='yield', columns='flight_op_carrier')
            index_tmp = index_tmp.fillna(method='bfill')
            index_tmp = index_tmp.fillna(0)
            index_tmp = index_tmp.rolling(28).mean()

            df_tmp = pd.DataFrame(index_tmp.stack(level=0))

            airline_index_tmp = pd.DataFrame()
            airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
            airline_index_tmp["Region"] = region
            airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
            airline_index_tmp["yield"] = list(df_tmp[df_tmp.columns[0]])
            airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)



Asia/Pacific-CIS
Africa-Latin America
Asia/Pacific-Latin America
Middle East-Middle East
Europe-Europe
CIS-CIS
Africa-Africa
Asia/Pacific-North America
Europe-Latin America
Africa-CIS
Asia/Pacific-Middle East
CIS-Middle East
Latin America-Middle East
CIS-North America
CIS-Latin America
Asia/Pacific-Europe
Latin America-Latin America
Asia/Pacific-Asia/Pacific
None
Middle East-North America
Africa-Middle East
CIS-Europe
Latin America-North America
Europe-Middle East
North America-North America
Africa-Europe
Europe-North America
Africa-North America
Africa-Asia/Pacific


In [714]:
airline_index

""


In [716]:
%%bigquery df_0
SELECT dt_of_issue as Date, region_pair as Region, flight_op_carrier as  operating_carrier, yield 
FROM `skytra-benchmark-rnd.products_IN1139.P002_Revenue_yield_per_region_pair_airline` 
WHERE yield > 0

In [717]:
df_0

,Date,Region,operating_carrier,yield
0,2018-03-05,Africa-Africa,2J,0.316201
1,2018-10-21,Africa-Africa,2J,0.338552
2,2018-11-16,Africa-Africa,2J,0.352578
3,2018-12-05,Africa-Africa,2J,0.368122
4,2018-11-09,Africa-Africa,3O,0.323692
...,...,...,...,...
2132729,2018-07-20,North America-North America,YN,0.189370
2132730,2020-05-04,North America-North America,YN,0.174729
2132731,2018-03-04,North America-North America,YP,0.382699
2132732,2018-08-12,North America-North America,YP,0.205661


In [647]:
df_0.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_yield_per_region',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

1it [00:43, 43.50s/it]


In [653]:
airline_index = pd.DataFrame()

for region in regions:
    try:
            tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
            tmp_table["yield"] = tmp_table["yield"].astype('float')
            index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='yield', columns='flight_op_carrier')
            index_tmp = index_tmp.fillna(method='bfill')
            index_tmp = index_tmp.fillna(0)
            index_tmp = index_tmp.rolling(7).mean()
            index_tmp = index_tmp.pct_change(365)

            df_tmp = pd.DataFrame(index_tmp.stack(level=0))

            airline_index_tmp = pd.DataFrame()
            airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
            airline_index_tmp["Region"] = region
            airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
            airline_index_tmp["yield"] = list(df_tmp[df_tmp.columns[0]])
            airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


None


In [654]:
airline_index

,Date,Region,operating_carrier,yield
0,2018-10-25,Africa-Asia/Pacific,2J,-0.214883
1,2018-10-25,Africa-Asia/Pacific,3J,0.132269
2,2018-10-25,Africa-Asia/Pacific,3K,-0.090153
3,2018-10-25,Africa-Asia/Pacific,3U,-0.461068
4,2018-10-25,Africa-Asia/Pacific,4D,-1.000000
...,...,...,...,...
81962,2020-05-27,Asia/Pacific-Middle East,WK,-1.000000
81963,2020-05-27,Asia/Pacific-Middle East,WY,-0.110481
81964,2020-05-27,Asia/Pacific-Middle East,XQ,-1.000000
81965,2020-05-27,Asia/Pacific-Middle East,XY,-0.298797


In [655]:


airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_yield_per_region_return',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

1it [00:32, 32.26s/it]


In [ ]:
%%bigquery df_0
SELECT
  country_pair,
  flight_op_carrier,
  dt_of_issue,
  SUM(leg_revenue)/(SUM(dist_km)) AS yield
FROM
  `skytra-benchmark-rnd.matching.R6`
WHERE
  transaction_code = "I"   and dt_of_issue > "2018-12-31"
GROUP BY
  country_pair,
  flight_op_carrier,
  dt_of_issue

In [ ]:
# The main IATA index dataframe
skytra_sf_df= df_0
skytra_sf_df.shape




In [ ]:
skytra_sf_df = skytra_sf_df[skytra_sf_df["dt_of_issue"]  < pd.to_datetime(end_date)]

In [ ]:
skytra_sf_df["country_pair"] = np.where(skytra_sf_df["country_pair"].isnull(),
                                       "multiple",
                                       skytra_sf_df["country_pair"])

skytra_sf_df = skytra_sf_df[skytra_sf_df["country_pair"] != "multiple"]



In [ ]:
countries = pd.DataFrame()
countries['country_pair'] = skytra_sf_df.country_pair
p = pd.DataFrame([countries['country_pair'].value_counts()>5000]).T
countries = list(p.head(250).index)

In [ ]:
airline_index = pd.DataFrame()

for region in regions:
    try:
        tmp_table = skytra_sf_df[skytra_sf_df["country_pair"] == region]
        index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='yield', columns='flight_op_carrier')
        index_tmp = index_tmp.fillna(method='bfill')
        index_tmp = index_tmp.fillna(0)
        index_tmp = index_tmp.rolling(28).mean()
    
        df_tmp = pd.DataFrame(index_tmp.stack(level=0))
    
        airline_index_tmp = pd.DataFrame()
        airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
        airline_index_tmp["Region"] = region
        airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
        airline_index_tmp["yield"] = list(df_tmp[df_tmp.columns[0]])
        airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)



In [ ]:
airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_yield_per_country_pair',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
region

# Creating revenue indices - Airline

In [656]:
%%bigquery df_0
SELECT
  dt_of_issue,
  region_pair, 
  flight_op_carrier as operating_carrier,
  SUM(
  IF
    (transaction_code = "I",
      leg_revenue,
      0))/1000000 AS revenue,
  SUM(
  IF
    (transaction_code = "I",
      dist_km,
      0))/1000000 AS rpk,
      SUM(
  IF
    (transaction_code = "R",
      dist_km,
      0))/1000000 AS rpk_refund,
  SUM(
  IF
    (transaction_code = "R",
      leg_revenue,
      0))/1000000 AS revenue_refund
FROM
  `skytra-benchmark-rnd.matching.R6`
WHERE dt_of_issue > "2018-12-31"
GROUP BY
  dt_of_issue,
  region_pair,
  flight_op_carrier

In [657]:
# The main IATA index dataframe
skytra_sf_df= df_0
skytra_sf_df.tail()
skytra_sf_df = skytra_sf_df[skytra_sf_df["dt_of_issue"]  < pd.to_datetime(end_date)]
skytra_sf_df["Net_revenue"] = skytra_sf_df["revenue"] - skytra_sf_df["revenue_refund"]
skytra_sf_df["Net_rpk"] = skytra_sf_df["rpk"] - skytra_sf_df["rpk_refund"]


In [658]:
skytra_sf_df.head()

,dt_of_issue,region_pair,operating_carrier,revenue,rpk,rpk_refund,revenue_refund,Net_revenue,Net_rpk
0,2020-04-01,"Asia/Pacific-Asia/Pacific,Asia/Pacific-North A...",AC,0E-9,0E-9,0.024407800,0.002258810,-0.002258810,-0.024407800
1,2019-11-03,"Asia/Pacific-Middle East,Middle East-North Ame...",EK,0.094424590,0.870480100,0.136350700,0.015476970,0.078947620,0.734129400
2,2019-11-03,"Europe-Latin America,Europe-Middle East",AF,0.002766900,0.023232000,0.008935200,0.001160080,0.001606820,0.014296800
3,2019-11-03,"Africa-Europe,Africa-Middle East,Europe-Middle...",LH,0.000432030,0.003994800,0E-9,0E-9,0.000432030,0.003994800
4,2019-09-02,"Europe-Latin America,Europe-Middle East,Latin ...",KL,0.001645030,0.014796200,0E-9,0E-9,0.001645030,0.014796200


In [659]:
skytra_sf_df.region_pair.unique()

array(['Asia/Pacific-Asia/Pacific,Asia/Pacific-North America,North America-North America',
       'Asia/Pacific-Middle East,Middle East-North America',
       'Europe-Latin America,Europe-Middle East', ...,
       'Asia/Pacific-Europe,Asia/Pacific-North America,Europe-North America',
       'Europe-Latin America,Europe-North America,Latin America-North America',
       'Asia/Pacific-Latin America,Asia/Pacific-North America,Latin America-North America'],
      dtype=object)

In [660]:
skytra_sf_df = skytra_sf_df[skytra_sf_df['region_pair'].isin(regions)] 
skytra_sf_df

,dt_of_issue,region_pair,operating_carrier,revenue,rpk,rpk_refund,revenue_refund,Net_revenue,Net_rpk
633029,2019-05-04,CIS-CIS,SU,4.492658310,62.032831500,2.731678000,0.294656270,4.198002040,59.301153500
633030,2019-09-22,CIS-CIS,M9,0.002445970,0.010333600,0E-9,0E-9,0.002445970,0.010333600
633031,2019-11-28,CIS-CIS,S7,4.582908200,68.524333600,2.243328400,0.232004570,4.350903630,66.281005200
633032,2019-07-27,CIS-CIS,B2,0.129495350,0.983816800,0.041988400,0.005789310,0.123706040,0.941828400
633033,2019-11-03,CIS-CIS,7W,0.002879960,0.007149700,0E-9,0E-9,0.002879960,0.007149700
...,...,...,...,...,...,...,...,...,...
1634684,2019-03-03,North America-North America,YN,0.000542920,0.004559000,0E-9,0E-9,0.000542920,0.004559000
1634685,2019-07-27,North America-North America,PD,0.177989810,0.720068100,0.002785600,0.000354680,0.177635130,0.717282500
1634686,2019-08-08,North America-North America,7F,0.115344930,0.266901000,0.017625000,0.003899870,0.111445060,0.249276000
1634687,2019-04-28,North America-North America,LF,0.010523910,0.044041100,0E-9,0E-9,0.010523910,0.044041100


In [661]:
tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
tmp_table


,dt_of_issue,region_pair,operating_carrier,revenue,rpk,rpk_refund,revenue_refund,Net_revenue,Net_rpk
1265639,2019-05-11,Asia/Pacific-Middle East,3U,0.044133600,0.463471600,0E-9,0E-9,0.044133600,0.463471600
1265640,2019-05-11,Asia/Pacific-Middle East,TK,0.563208040,6.381944100,0.357227200,0.046517180,0.516690860,6.024716900
1265641,2019-11-02,Asia/Pacific-Middle East,ET,0.025156460,0.384261000,0.005965400,0.000633750,0.024522710,0.378295600
1265642,2020-03-01,Asia/Pacific-Middle East,UL,0.861205000,14.005100400,4.087290200,0.398224150,0.462980850,9.917810200
1265643,2019-07-15,Asia/Pacific-Middle East,EK,9.998288220,89.220926000,5.630540200,0.725072250,9.273215970,83.590385800
...,...,...,...,...,...,...,...,...,...
1304937,2019-03-20,Asia/Pacific-Middle East,ZH,0.000344010,0.002920000,0E-9,0E-9,0.000344010,0.002920000
1304938,2019-05-26,Asia/Pacific-Middle East,BL,0.000119020,0.001160200,0E-9,0E-9,0.000119020,0.001160200
1304939,2019-12-16,Asia/Pacific-Middle East,8M,0.000031060,0.000611100,0E-9,0E-9,0.000031060,0.000611100
1304940,2019-02-12,Asia/Pacific-Middle East,LH,0.001250180,0.009358600,0E-9,0E-9,0.001250180,0.009358600


In [667]:
airline_index = pd.DataFrame()

for region in regions:
    try:
        tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
        tmp_table["revenue"] = tmp_table["revenue"].astype(float)
        index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='revenue', columns='operating_carrier')
        index_tmp = index_tmp.fillna(0)
        index_tmp = index_tmp.rolling(7).sum()
        index_tmp = index_tmp.pct_change(365)
        

        df_tmp = pd.DataFrame(index_tmp.stack(level=0))

        airline_index_tmp = pd.DataFrame()
        airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
        airline_index_tmp["Region"] = region
        airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
        airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
        airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


None


In [668]:
airline_index = airline_index[~airline_index.isin([np.nan, np.inf, -np.inf]).any(1)] 
airline_index.head()

,Date,Region,operating_carrier,revenue
0,2020-01-07,Africa-Asia/Pacific,3K,0.527698
1,2020-01-07,Africa-Asia/Pacific,3U,0.628886
5,2020-01-07,Africa-Asia/Pacific,9W,-1.000000
7,2020-01-07,Africa-Asia/Pacific,AF,-0.197940
8,2020-01-07,Africa-Asia/Pacific,AH,-0.533746


In [669]:
airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_revenue_per_region_return',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

1it [00:09,  9.77s/it]


In [ ]:
airline_index = pd.DataFrame()

for region in regions:
    try:
        tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
        tmp_table["rpk"] = tmp_table["rpk"].astype(float)
        tmp_table["Net_rpk"] = tmp_table["Net_rpk"].astype(float)
        index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='Net_rpk', columns='operating_carrier')
        index_tmp = index_tmp.fillna(0)
        index_tmp = index_tmp.rolling(7).sum()
        index_tmp = index_tmp.pct_change(365)

        df_tmp = pd.DataFrame(index_tmp.stack(level=0))

        airline_index_tmp = pd.DataFrame()
        airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
        airline_index_tmp["Region"] = region
        airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
        airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
        airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)



In [ ]:
airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_Net_rpk_per_region_return',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
skytra_sf_df.head()

In [ ]:
countries = skytra_sf_df["country_pair"].unique()
countries

In [ ]:
airline_index = pd.DataFrame()

for region in countries:
    try:
        tmp_table = skytra_sf_df[skytra_sf_df["country_pair"] == region]
        index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='revenue', columns='operating_carrier')
        index_tmp = index_tmp.fillna(0)
        index_tmp = index_tmp.rolling(7).sum()
        index_tmp = index_tmp.pct_change(365)

        df_tmp = pd.DataFrame(index_tmp.stack(level=0))

        airline_index_tmp = pd.DataFrame()
        airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
        airline_index_tmp["Region"] = region
        airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
        airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
        airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)



In [ ]:
airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_revenue_per_country_return',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
airline_index = pd.DataFrame()

for region in regions:
    try:
            tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
            index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='revenue', columns='operating_carrier')
            index_tmp = index_tmp.fillna(0)
            index_tmp = index_tmp.rolling(7).sum()
            index_tmp = index_tmp

            df_tmp = pd.DataFrame(index_tmp.stack(level=0))

            airline_index_tmp = pd.DataFrame()
            airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
            airline_index_tmp["Region"] = region
            airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
            airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
            airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)



In [ ]:
airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_revenue_per_region_USD',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
airline_index = pd.DataFrame()

for region in regions:
    try:
            tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
            tmp_table["rpk"] = tmp_table["rpk"].astype(float)
            index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='rpk', columns='operating_carrier')
            index_tmp = index_tmp.fillna(0)
            index_tmp = index_tmp.rolling(7).sum()
            index_tmp = index_tmp.pct_change(365)

            df_tmp = pd.DataFrame(index_tmp.stack(level=0))

            airline_index_tmp = pd.DataFrame()
            airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
            airline_index_tmp["Region"] = region
            airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
            airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
            airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)



In [ ]:
airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_rpk_per_region_return',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
tmp_table["rpk"] = tmp_table["rpk"].astype(float)

In [ ]:
airline_index = pd.DataFrame()

for region in regions:
    try:
        tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
        tmp_table["rpk"] = tmp_table["rpk"].astype(float)
        index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='rpk', columns='operating_carrier')
        
        index_tmp = index_tmp.fillna(0)
        index_tmp = index_tmp.rolling(7).sum()
        index_tmp = index_tmp

        df_tmp = pd.DataFrame(index_tmp.stack(level=0))

        airline_index_tmp = pd.DataFrame()
        airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
        airline_index_tmp["Region"] = region
        airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
        airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
        airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)
    


In [ ]:
airline_index

In [ ]:
airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_revenue_per_region_rpk',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
airline_index = pd.DataFrame()

for region in regions:
    print(region)
    try:
        tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
        tmp_table["rpk"] = tmp_table["rpk"].astype(float)
        tmp_table["Net_rpk"] = tmp_table["Net_rpk"].astype(float)
        index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='Net_rpk', columns='operating_carrier')
        index_tmp = index_tmp.fillna(0)
        index_tmp = index_tmp.rolling(7).sum()
        index_tmp = index_tmp

        df_tmp = pd.DataFrame(index_tmp.stack(level=0))

        airline_index_tmp = pd.DataFrame()
        airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
        airline_index_tmp["Region"] = region
        airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
        airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
        airline_index = airline_index.append(airline_index_tmp)
        print("ok")
        print(region)
    except:
        print("not ok")
        print(region)
    


In [ ]:
airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_revenue_per_region_netrpk',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
airline_index = pd.DataFrame()

for region in regions:
    try:
            tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
            tmp_table["Net_revenue"] = tmp_table["Net_revenue"].astype(float)
            index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='Net_revenue', columns='operating_carrier')
            index_tmp = index_tmp.fillna(0)
            index_tmp = index_tmp.rolling(7).sum()
            index_tmp = index_tmp.pct_change(365)

            df_tmp = pd.DataFrame(index_tmp.stack(level=0))

            airline_index_tmp = pd.DataFrame()
            airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
            airline_index_tmp["Region"] = region
            airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
            airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
            airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)


In [ ]:
airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_netrevenue_per_region_return',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
airline_index = pd.DataFrame()

for region in regions:
    try:
            tmp_table = skytra_sf_df[skytra_sf_df["region_pair"] == region]
            tmp_table["Net_revenue"] = tmp_table["Net_revenue"].astype(float)
            index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='Net_revenue', columns='operating_carrier')
            index_tmp = index_tmp.fillna(0)
            index_tmp = index_tmp.rolling(7).sum()
            index_tmp = index_tmp

            df_tmp = pd.DataFrame(index_tmp.stack(level=0))

            airline_index_tmp = pd.DataFrame()
            airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
            airline_index_tmp["Region"] = region
            airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
            airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
            airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)



In [ ]:
airline_index["Date"].max()

In [ ]:
airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_netrevenue_per_region_USD',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

# Creating forward curves - Airline

In [718]:
%%bigquery df_0
SELECT
  dt_of_issue,
  lead_bucket,
  region_pair,
  op_carrier,
  SUM(revenue) AS revenue, SUM(rpk) AS rpk, SUM(net_revenue) AS net_revenue, SUM(net_rpk) AS net_rpk
FROM
  `skytra-benchmark-rnd.matching.R7_itineraries`
WHERE
  dt_of_issue > "2018-12-10"
  AND op_carrier IN ("AA",
    "BA",
    "DL",
    "LH",
    "MU",
    "CI",
    "CX",
    "MU",
    "AF",
    "UA",
    "AF", "AY")
  AND region_pair IN ("Europe-Europe",
    "North America-North America",
    "Europe-North America",
    "Asia/Pacific-Asia/Pacific",
    "Asia/Pacific-Europe",
    "Asia/Pacific-North America")
GROUP BY
  dt_of_issue,
  lead_bucket,
  region_pair,
  op_carrier

In [719]:
forward_revenue = df_0
forward_revenue.head()

,dt_of_issue,lead_bucket,region_pair,op_carrier,revenue,rpk,net_revenue,net_rpk
0,2019-02-28,211-240 days,Asia/Pacific-North America,AA,72165.830000000,883331.000000000,68621.110000000,833688.000000000
1,2020-01-19,121-150 days,Asia/Pacific-North America,AA,331613.390000000,3370439.000000000,252169.940000000,2373518.000000000
2,2020-02-28,241-270 days,North America-North America,AA,468075.290000000,3466896.000000000,461181.580000000,3397532.000000000
3,2020-02-27,<= 30 days,Asia/Pacific-North America,AA,979873.580000000,11042307.000000000,528846.310000000,5784288.000000000
4,2020-02-29,91-120 days,Europe-North America,AA,1085215.620000000,8730748.000000000,849987.870000000,6821502.000000000


In [720]:
forward_revenue = forward_revenue[forward_revenue["dt_of_issue"]  < pd.to_datetime(end_date)]

In [721]:
forward_revenue["Net_revenue"] = forward_revenue["net_revenue"] 
forward_revenue["Net_rpk"] = forward_revenue["net_rpk"] 
forward_revenue["period"] = forward_revenue["lead_bucket"]

In [722]:
forward_revenue

,dt_of_issue,lead_bucket,region_pair,op_carrier,revenue,rpk,net_revenue,net_rpk,Net_revenue,Net_rpk,period
0,2019-02-28,211-240 days,Asia/Pacific-North America,AA,72165.830000000,883331.000000000,68621.110000000,833688.000000000,68621.110000000,833688.000000000,211-240 days
1,2020-01-19,121-150 days,Asia/Pacific-North America,AA,331613.390000000,3370439.000000000,252169.940000000,2373518.000000000,252169.940000000,2373518.000000000,121-150 days
2,2020-02-28,241-270 days,North America-North America,AA,468075.290000000,3466896.000000000,461181.580000000,3397532.000000000,461181.580000000,3397532.000000000,241-270 days
3,2020-02-27,<= 30 days,Asia/Pacific-North America,AA,979873.580000000,11042307.000000000,528846.310000000,5784288.000000000,528846.310000000,5784288.000000000,<= 30 days
4,2020-02-29,91-120 days,Europe-North America,AA,1085215.620000000,8730748.000000000,849987.870000000,6821502.000000000,849987.870000000,6821502.000000000,91-120 days
...,...,...,...,...,...,...,...,...,...,...,...
218545,2020-03-05,331-360 days,North America-North America,UA,1276.970000000,11373.000000000,1276.970000000,11373.000000000,1276.970000000,11373.000000000,331-360 days
218546,2020-02-12,151-180 days,Asia/Pacific-Asia/Pacific,UA,84996.950000000,377942.000000000,80985.690000000,352792.000000000,80985.690000000,352792.000000000,151-180 days
218547,2020-02-12,<= 30 days,North America-North America,UA,53686080.220000000,361771006.000000000,52395662.230000000,352852389.000000000,52395662.230000000,352852389.000000000,<= 30 days
218548,2019-10-17,151-180 days,Europe-North America,UA,767839.730000000,8574010.000000000,733386.520000000,8033567.000000000,733386.520000000,8033567.000000000,151-180 days


In [723]:
regions = forward_revenue.region_pair.unique()
regions

array(['Asia/Pacific-North America', 'North America-North America',
       'Europe-North America', 'Asia/Pacific-Europe', 'Europe-Europe',
       'Asia/Pacific-Asia/Pacific'], dtype=object)

In [724]:
airline_forward = pd.DataFrame()

for region in regions: 
    regional_table = forward_revenue[forward_revenue["region_pair"] == region] 
    
    for period in forward_revenue.period.unique():
        try:
            tmp_table = regional_table[regional_table.period== period]
            tmp_table["revenue"] = tmp_table["revenue"].astype('float64')
            index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='revenue', columns='op_carrier')
            index_tmp = index_tmp.fillna(0)
            index_tmp = index_tmp.rolling(7).sum()
            index_tmp = index_tmp.pct_change(365)


            df_tmp = pd.DataFrame(index_tmp.stack(level=0))

            airline_index_tmp = pd.DataFrame()
            airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
            airline_index_tmp["Region"] = region
            airline_index_tmp["period"] = period
            airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
            airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
            airline_forward = airline_forward.append(airline_index_tmp)
        except:
            print("not ok")
            print(region)
    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [725]:
airline_forward

,Date,Region,period,operating_carrier,revenue
0,2019-12-17,Asia/Pacific-North America,211-240 days,AA,-0.039534
1,2019-12-17,Asia/Pacific-North America,211-240 days,AF,-0.871229
2,2019-12-17,Asia/Pacific-North America,211-240 days,AY,-inf
3,2019-12-17,Asia/Pacific-North America,211-240 days,BA,-0.180351
4,2019-12-17,Asia/Pacific-North America,211-240 days,CI,1.341835
...,...,...,...,...,...
934,2020-06-03,Asia/Pacific-Asia/Pacific,301-330 days,CI,-0.883688
935,2020-06-03,Asia/Pacific-Asia/Pacific,301-330 days,CX,-0.734370
936,2020-06-03,Asia/Pacific-Asia/Pacific,301-330 days,DL,0.500000
937,2020-06-03,Asia/Pacific-Asia/Pacific,301-330 days,MU,-0.757767


In [726]:
airline_forward.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_forward_yoy',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

1it [00:03,  3.85s/it]


In [727]:
airline_forward = pd.DataFrame()

for region in regions: 
    regional_table = forward_revenue[forward_revenue["region_pair"] == region] 
    
    for period in forward_revenue.period.unique():
        try:
            tmp_table = regional_table[regional_table.period== period]
            tmp_table = forward_revenue[forward_revenue.period== period]
            tmp_table["revenue"] = tmp_table["revenue"].astype('float64')
            index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='revenue', columns='op_carrier')
            index_tmp = index_tmp.fillna(0)
            index_tmp = index_tmp.rolling(7).sum()
            index_tmp = index_tmp


            df_tmp = pd.DataFrame(index_tmp.stack(level=0))

            airline_index_tmp = pd.DataFrame()
            airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
            airline_index_tmp["Region"] = region
            airline_index_tmp["period"] = period
            airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
            airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
            airline_forward = airline_forward.append(airline_index_tmp)
        except:
            print("not ok")
            print("region")
        
    print(region)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Asia/Pacific-North America
North America-North America
Europe-North America
Asia/Pacific-Europe
Europe-Europe
Asia/Pacific-Asia/Pacific


In [728]:
airline_forward.head()

,Date,Region,period,operating_carrier,revenue
0,2018-12-17,Asia/Pacific-North America,211-240 days,AA,1.251449e+06
1,2018-12-17,Asia/Pacific-North America,211-240 days,AF,8.340858e+05
2,2018-12-17,Asia/Pacific-North America,211-240 days,AY,4.182877e+05
3,2018-12-17,Asia/Pacific-North America,211-240 days,BA,6.549135e+05
4,2018-12-17,Asia/Pacific-North America,211-240 days,CI,1.917307e+05


In [729]:
airline_forward.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_forward_USD',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

1it [00:08,  8.12s/it]


In [730]:
airline_forward = pd.DataFrame()

for region in regions: 
    regional_table = forward_revenue[forward_revenue["region_pair"] == region] 
    
    for period in forward_revenue.period.unique():
        try:
            tmp_table = regional_table[regional_table.period== period]
            tmp_table = forward_revenue[forward_revenue.period== period]
            tmp_table["rpk"] = tmp_table["rpk"].astype('float64')
            index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='rpk', columns='op_carrier')
            index_tmp = index_tmp.fillna(0)
            index_tmp = index_tmp.rolling(7).sum()
            index_tmp = index_tmp


            df_tmp = pd.DataFrame(index_tmp.stack(level=0))

            airline_index_tmp = pd.DataFrame()
            airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
            airline_index_tmp["Region"] = region
            airline_index_tmp["period"] = period
            airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
            airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
            airline_forward = airline_forward.append(airline_index_tmp)
        except:
            print("not ok")
            print("region")
        
    print(region)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Asia/Pacific-North America
North America-North America
Europe-North America
Asia/Pacific-Europe
Europe-Europe
Asia/Pacific-Asia/Pacific


In [731]:
airline_forward.head()

,Date,Region,period,operating_carrier,revenue
0,2018-12-17,Asia/Pacific-North America,211-240 days,AA,1.172142e+07
1,2018-12-17,Asia/Pacific-North America,211-240 days,AF,6.219151e+06
2,2018-12-17,Asia/Pacific-North America,211-240 days,AY,4.183595e+06
3,2018-12-17,Asia/Pacific-North America,211-240 days,BA,5.868034e+06
4,2018-12-17,Asia/Pacific-North America,211-240 days,CI,1.940429e+06


In [732]:
airline_forward.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_forward_rpk',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

1it [00:10, 10.48s/it]


In [733]:
airline_forward = pd.DataFrame()

for period in forward_revenue.period.unique():
    try:
            tmp_table = forward_revenue[forward_revenue.period== period]
            tmp_table["revenue"] = tmp_table["revenue"].astype(float)
            index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='revenue', columns='region_pair')
            index_tmp = index_tmp.fillna(0)
            index_tmp = index_tmp.rolling(7).sum()
            index_tmp = index_tmp.pct_change(365)


            df_tmp = pd.DataFrame(index_tmp.stack(level=0))

            airline_index_tmp = pd.DataFrame()
            airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
            airline_index_tmp["Region"] = "Europe-Europe"
            airline_index_tmp["period"] = period
            airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
            airline_forward = airline_forward.append(airline_index_tmp)
    except:
        print(period)


211-240 days
121-150 days
241-270 days
<= 30 days
91-120 days
61-90 days
151-180 days
271-300 days
31-60 days
181-210 days
331-360 days
301-330 days
> 360 days


In [734]:
airline_forward.to_gbq(destination_table='IN1092_new_products_dashboard.Region_forward_yoy',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [735]:
airline_forward = pd.DataFrame()

for region in regions: 
    regional_table = forward_revenue[forward_revenue["region_pair"] == region] 
    
    for period in forward_revenue.period.unique():
        try:
                    tmp_table = regional_table[regional_table.period== period]
                    tmp_table["Net_revenue"] = tmp_table["Net_revenue"].astype(float)
                
                    index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='Net_revenue', columns='op_carrier')
                    index_tmp = index_tmp.fillna(0)
                    index_tmp = index_tmp.rolling(7).sum()
                    index_tmp = index_tmp.pct_change(365)


                    df_tmp = pd.DataFrame(index_tmp.stack(level=0))

                    airline_index_tmp = pd.DataFrame()
                    airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
                    airline_index_tmp["Region"] = region
                    airline_index_tmp["period"] = period
                    airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
                    airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
                    airline_forward = airline_forward.append(airline_index_tmp)
        except:
            print("bad")
            print(region)

    print(region)

bad
Asia/Pacific-North America
bad
Asia/Pacific-North America
bad
Asia/Pacific-North America
bad
Asia/Pacific-North America
bad
Asia/Pacific-North America
bad
Asia/Pacific-North America
bad
Asia/Pacific-North America
bad
Asia/Pacific-North America
bad
Asia/Pacific-North America
bad
Asia/Pacific-North America
bad
Asia/Pacific-North America
bad
Asia/Pacific-North America
bad
Asia/Pacific-North America
Asia/Pacific-North America
bad
North America-North America
bad
North America-North America
bad
North America-North America
bad
North America-North America
bad
North America-North America
bad
North America-North America
bad
North America-North America
bad
North America-North America
bad
North America-North America
bad
North America-North America
bad
North America-North America
bad
North America-North America
bad
North America-North America
North America-North America
bad
Europe-North America
bad
Europe-North America
bad
Europe-North America
bad
Europe-North America
bad
Europe-North America
b

In [736]:
airline_forward.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_forward_net_revenue_yoy',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [738]:
airline_forward = pd.DataFrame()

for region in regions: 
    regional_table = forward_revenue[forward_revenue["region_pair"] == region] 
    
    for period in forward_revenue.period.unique():
        tmp_table = regional_table[regional_table.period== period]
        tmp_table = forward_revenue[forward_revenue.period== period]
        tmp_table["Net_revenue"] = tmp_table["Net_revenue"].astype(float)
        index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='Net_revenue', columns='op_carrier')
        index_tmp = index_tmp.fillna(0)
        index_tmp = index_tmp.rolling(7).sum()
        index_tmp = index_tmp


        df_tmp = pd.DataFrame(index_tmp.stack(level=0))

        airline_index_tmp = pd.DataFrame()
        airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
        airline_index_tmp["Region"] = region
        airline_index_tmp["period"] = period
        airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
        airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
        airline_forward = airline_forward.append(airline_index_tmp)
    print(region)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Asia/Pacific-North America
North America-North America
Europe-North America
Asia/Pacific-Europe
Europe-Europe
Asia/Pacific-Asia/Pacific


In [739]:
airline_forward.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_forward_net_revenue_USD',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

1it [00:07,  7.08s/it]


In [740]:
%%bigquery df_0
SELECT
  usd.*,
  rpk.revenue AS rpk, safe_divide(usd.revenue,rpk.revenue) as yield
FROM
  `skytra-benchmark-rnd.IN1092_new_products_dashboard.Airline_forward_USD` usd
LEFT JOIN
  `skytra-benchmark-rnd.IN1092_new_products_dashboard.Airline_forward_rpk` rpk
ON
  usd.Date = rpk.Date
  AND usd.Region = rpk.Region
  AND usd.period = rpk.period
  AND usd.operating_carrier = rpk.operating_carrier

In [741]:
df_0 = df_0.fillna(method='ffill')
df_0.head()

,Date,Region,period,operating_carrier,revenue,rpk,yield
0,2018-12-17,Asia/Pacific-North America,211-240 days,AA,1.251449e+06,1.172142e+07,0.106766
1,2018-12-18,Asia/Pacific-North America,211-240 days,AA,1.208282e+06,1.130299e+07,0.106899
2,2018-12-19,Asia/Pacific-North America,211-240 days,AA,1.214904e+06,1.135228e+07,0.107019
3,2018-12-20,Asia/Pacific-North America,211-240 days,AA,1.201897e+06,1.135967e+07,0.105804
4,2018-12-21,Asia/Pacific-North America,211-240 days,AA,1.201622e+06,1.140166e+07,0.105390


In [742]:
df_0 = df_0[df_0["period"] != '> 360 days']
df_0.head()

,Date,Region,period,operating_carrier,revenue,rpk,yield
0,2018-12-17,Asia/Pacific-North America,211-240 days,AA,1.251449e+06,1.172142e+07,0.106766
1,2018-12-18,Asia/Pacific-North America,211-240 days,AA,1.208282e+06,1.130299e+07,0.106899
2,2018-12-19,Asia/Pacific-North America,211-240 days,AA,1.214904e+06,1.135228e+07,0.107019
3,2018-12-20,Asia/Pacific-North America,211-240 days,AA,1.201897e+06,1.135967e+07,0.105804
4,2018-12-21,Asia/Pacific-North America,211-240 days,AA,1.201622e+06,1.140166e+07,0.105390


In [743]:
df_0.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_forward_yield',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

1it [00:10, 10.24s/it]


# Country revenues

In [744]:
%%bigquery df_0
SELECT
  dt_of_issue,
  country_pair, 
  SUM(
  IF
    (transaction_code = "I",
      leg_revenue,
      0))/1000000 AS revenue,
  SUM(
  IF
    (transaction_code = "I",
      dist_km,
      0))/1000000 AS rpk,
      SUM(
  IF
    (transaction_code = "R",
      dist_km,
      0))/1000000 AS rpk_refund,
  SUM(
  IF
    (transaction_code = "R",
      leg_revenue,
      0))/1000000 AS revenue_refund
FROM
  `skytra-benchmark-rnd.matching.R6`
WHERE dt_of_issue > "2018-12-31" and   country_pair LIKE "%Korea%"
  OR country_pair LIKE "%China%"
  OR country_pair LIKE "%Sweden%"
  OR country_pair LIKE "%Japan%"
  OR country_pair LIKE "%Germany%"
GROUP BY
  dt_of_issue,
  country_pair

In [745]:
# The main IATA index dataframe
skytra_sf_df= df_0
skytra_sf_df.tail()
skytra_sf_df = skytra_sf_df[skytra_sf_df["dt_of_issue"]  < pd.to_datetime(end_date)]
skytra_sf_df["Net_revenue"] = skytra_sf_df["revenue"] - skytra_sf_df["revenue_refund"]
skytra_sf_df["Net_rpk"] = skytra_sf_df["rpk"] - skytra_sf_df["rpk_refund"]
skytra_sf_df.head()

,dt_of_issue,country_pair,revenue,rpk,rpk_refund,revenue_refund,Net_revenue,Net_rpk
0,2020-02-26,Serbia-Sweden,0.013621280,0.154712600,0.004986900,0.000221920,0.013399360,0.149725700
1,2020-02-26,Germany-Singapore,0.213153270,3.080523400,1.706410300,0.109060840,0.104092430,1.374113100
2,2020-02-26,"China-Germany,China-Norway",0E-9,0E-9,0.020084700,0.001398650,-0.001398650,-0.020084700
3,2020-02-26,"Hong Kong-South Korea,Netherlands-South Korea",0E-9,0E-9,0.011811000,0.001274340,-0.001274340,-0.011811000
4,2020-02-26,"Indonesia-Japan,Japan-Singapore",0E-9,0E-9,0.056523400,0.005851220,-0.005851220,-0.056523400


In [746]:
skytra_sf_df = skytra_sf_df[-(skytra_sf_df.country_pair.str.contains(',', case=True, regex=True))]
skytra_sf_df.head()

,dt_of_issue,country_pair,revenue,rpk,rpk_refund,revenue_refund,Net_revenue,Net_rpk
0,2020-02-26,Serbia-Sweden,0.013621280,0.154712600,0.004986900,0.000221920,0.013399360,0.149725700
1,2020-02-26,Germany-Singapore,0.213153270,3.080523400,1.706410300,0.109060840,0.104092430,1.374113100
7,2018-07-26,China-Mauritius,0.183034510,1.280112100,0.036640200,0.005934860,0.177099650,1.243471900
9,2020-01-19,Azerbaijan-Germany,0.029464940,0.399010100,0.014140200,0.001047180,0.028417760,0.384869900
12,2018-06-19,Iran-Japan,0.045217290,0.324247100,0.040332000,0.005593180,0.039624110,0.283915100


In [747]:
regions = skytra_sf_df.country_pair.unique()
regions

array(['Serbia-Sweden', 'Germany-Singapore', 'China-Mauritius', ...,
       'Lebanon-North Korea', 'Dominica-South Korea',
       'Falkland Islands-Japan'], dtype=object)

In [748]:
region = regions[0]
tmp_table = skytra_sf_df[skytra_sf_df["country_pair"] == region]
index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='revenue', columns='country_pair')
index_tmp = index_tmp.fillna(0)
index_tmp = index_tmp.rolling(7).sum()
index_tmp = index_tmp.pct_change(365)

DataError: No numeric types to aggregate

In [ ]:
airline_index = pd.DataFrame()

for region in regions:
    try:
        tmp_table = skytra_sf_df[skytra_sf_df["country_pair"] == region]
        tmp_table["revenue"] = tmp_table["revenue"].astype(float)
        index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='revenue', columns='country_pair')
        index_tmp = index_tmp.fillna(0)
        index_tmp = index_tmp.rolling(7).sum()
        index_tmp = index_tmp.pct_change(365)

        df_tmp = pd.DataFrame(index_tmp.stack(level=0))

        airline_index_tmp = pd.DataFrame()
        airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
        airline_index_tmp["Region"] = region
        airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
        airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
        airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)



In [ ]:
airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_revenue_per_countrypair_return',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
airline_index = pd.DataFrame()

for region in regions:
    try:
            tmp_table = skytra_sf_df[skytra_sf_df["country_pair"] == region]
            tmp_table["revenue"] = tmp_table["revenue"].astype(float)
            index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='revenue', columns='country_pair')
            index_tmp = index_tmp.fillna(0)
            index_tmp = index_tmp.rolling(7).sum()
            index_tmp = index_tmp

            df_tmp = pd.DataFrame(index_tmp.stack(level=0))

            airline_index_tmp = pd.DataFrame()
            airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
            airline_index_tmp["Region"] = region
            airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
            airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
            airline_index = airline_index.append(airline_index_tmp)
    except:
        print(region)



In [ ]:
airline_index.head()

In [ ]:
airline_index.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_revenue_per_countrypair_USD',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
%%bigquery df_0
SELECT
  op_carrier,
  quarter_delivery,
  SUM(revenue) AS revenue
FROM
  `skytra-benchmark-rnd.matching.R7_itineraries`
WHERE
  dt_of_issue > "1900-05-18"
GROUP BY
  op_carrier,
  quarter_delivery

In [ ]:
# The main IATA index dataframe
skytra_sf_df= df_0
skytra_sf_df["revenue"] = skytra_sf_df["revenue"]/1000000
skytra_sf_df.head()

In [ ]:
skytra_sf_df.quarter_delivery.unique()

In [ ]:
skytra_sf_df.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_revenue_per_quarter_USD',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
%%bigquery df_0
SELECT
  dt_of_issue,
  month_delivery, region_pair,
  op_carrier,
  SUM(revenue) AS revenue, SUM(net_revenue) AS net_revenue
FROM
  `skytra-benchmark-rnd.matching.R7_itineraries`
WHERE month_delivery in ( "2020-08", "2019-08") and revenue > 0
GROUP BY
  dt_of_issue,region_pair,
  month_delivery,
  op_carrier

In [ ]:
# The main IATA index dataframe
skytra_sf_df= df_0
skytra_sf_df.tail()
skytra_sf_df = skytra_sf_df[skytra_sf_df["dt_of_issue"]  < pd.to_datetime(end_date)]

skytra_sf_df.head()

In [ ]:
regions = skytra_regions 

In [ ]:
skytra_sf_df.month_delivery.unique()

In [ ]:
airline_forward = pd.DataFrame()

for region in regions: 
    regional_table = skytra_sf_df[skytra_sf_df["region_pair"] == region] 
    
    for period in skytra_sf_df.month_delivery.unique():
        try:
            tmp_table = regional_table[regional_table.month_delivery== period]
            tmp_table["revenue"] = tmp_table["revenue"].astype(float)
            index_tmp = pd.pivot_table(tmp_table, index='dt_of_issue', values='revenue', columns='op_carrier')
            index_tmp = index_tmp.fillna(0)
            index_tmp = index_tmp.cumsum()
            index_tmp = index_tmp


            df_tmp = pd.DataFrame(index_tmp.stack(level=0))

            airline_index_tmp = pd.DataFrame()
            airline_index_tmp["Date"] = df_tmp.index.get_level_values(0)
            airline_index_tmp["Region"] = region
            airline_index_tmp["period"] = period
            airline_index_tmp["operating_carrier"] = df_tmp.index.get_level_values(1)
            airline_index_tmp["revenue"] = list(df_tmp[df_tmp.columns[0]])
            airline_forward = airline_forward.append(airline_index_tmp)
        except:
            print("not ok")
            print(region)
        
    print(region)

In [ ]:
airline_forward

In [ ]:
airline_forward[((airline_forward["Date"]  < pd.to_datetime("2019-09-01")) & (airline_forward["period"] == "2019-08"))]

In [ ]:


airline_forward[((airline_forward["Date"]  < pd.to_datetime("2019-09-01")) & (airline_forward["period"] == "2019-08"))].to_gbq(destination_table='IN1092_new_products_dashboard.Airline_revenue_future_month_USD',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
from datetime import datetime, timedelta

In [ ]:
airline_forward[((airline_forward["Date"]  > pd.to_datetime("2019-08-01")) & (airline_forward["period"] == "2020-08"))].to_gbq(destination_table='IN1092_new_products_dashboard.Airline_revenue_future_month_USD2',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
old_ts = airline_forward[((airline_forward["Date"]  < pd.to_datetime("2019-09-01")) & (airline_forward["period"] == "2019-08"))]
new_ts = airline_forward[((airline_forward["Date"]  > pd.to_datetime("2019-08-01")) & (airline_forward["period"] == "2020-08"))]

In [ ]:
airline = "BA"
region = "Europe-Europe"

In [ ]:
airline_list = ("BA", "DL", "AF", "UN", "CA", "CX", "MU", "AY")

In [ ]:
output_df = pd.DataFrame()
for airline in airline_list:
    for region in new_ts["Region"].unique():
        old_tmp = old_ts[ (old_ts["operating_carrier"] == airline) & (old_ts["Region"] == region) ]
        old_tmp["old_date"] = old_tmp["Date"].astype(str)
        new_tmp = new_ts[ (new_ts["operating_carrier"] == airline) & (new_ts["Region"] == region) ]
        
        merged_tmp = pd.DataFrame()
        merged_tmp["Date"] = new_tmp["Date"]
        merged_tmp["Region"] = new_tmp["Region"]
        merged_tmp["operating_carrier"] = new_tmp["operating_carrier"]
        merged_tmp["new_revenue"] = new_tmp["revenue"]
        merged_tmp["old_date"] = pd.to_datetime(merged_tmp["Date"] ) - timedelta(days=365)
        merged_tmp["old_date"] = merged_tmp["old_date"].astype(str)
        merged_tmp = pd.merge(merged_tmp, old_tmp[["old_date", "operating_carrier", "revenue" ]], how='left', on=['old_date', "operating_carrier"])
        merged_tmp = merged_tmp.fillna(method='ffill')
        merged_tmp["YoY"] = merged_tmp["new_revenue"]/merged_tmp["revenue"] -1 
        output_df = output_df.append(merged_tmp)
        print(region)
    

In [ ]:
output_df

In [ ]:
output_df.to_gbq(destination_table='IN1092_new_products_dashboard.Airline_august_yoy',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
new_tmp.head()

In [ ]:
%%bigquery df_0
WITH
  curent_year AS (
  SELECT
    CAST(year_delivery AS INT64) AS year_delivery,
    quarter, month,op_carrier,
    flyFrom_country,
    flyTo_country,
    flight_type,
    SUM(revenue) AS revenue, sum(rpk) as rpk
  FROM
    `skytra-benchmark-rnd.matching.R7_flights`
  GROUP BY
    year_delivery,
    quarter,month,op_carrier,
    flyFrom_country,
    flyTo_country,
    flight_type),
  previous_year AS (
  SELECT
    CAST(year_delivery AS INT64) + 1 AS year_delivery_md,
    quarter,month,op_carrier,
    flyFrom_country,
    flyTo_country,
    flight_type,
    SUM(revenue) AS revenue,sum(rpk) as rpk
  FROM
    `skytra-benchmark-rnd.matching.R7_flights`
  WHERE
    dt_of_issue < "2019-05-14"
  GROUP BY
    year_delivery,
    quarter,month,op_carrier,
    flyFrom_country,
    flyTo_country,
    flight_type)
SELECT
  c.* , IFNULL(p.revenue,0) as last_year_revenue, IFNULL(p.rpk,0) as last_year_rpk
FROM
  curent_year c
LEFT JOIN
  previous_year p
ON
  c.flyFrom_country = p.flyFrom_country
  AND c.flyTo_country = p.flyTo_country
  AND c.flight_type = p.flight_type
  AND c.year_delivery = p.year_delivery_md
  AND c.quarter = p.quarter
  AND c.month = p.month
  AND c.op_carrier = p.op_carrier
WHERE c.year_delivery is not null and c.year_delivery>2016


In [ ]:
df_0.head()

In [ ]:
df_0.to_gbq(destination_table='IN1092_new_products_dashboard.Country_per_quarter',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')

In [ ]:
%%bigquery df_0
WITH
  outbound_curent_year AS (
  SELECT
    CAST(year_delivery AS INT64) AS year_delivery,
    op_carrier,
    quarter,
    month,
    flyFrom_country AS country,
    flight_type,
    SUM(revenue) AS revenue,
    SUM(rpk) AS rpk
  FROM
    `skytra-benchmark-rnd.matching.R7_flights`
  GROUP BY
    year_delivery,
    quarter,
    month,
    flyFrom_country,
    op_carrier,
    flight_type),
  inbound_curent_year AS (
  SELECT
    CAST(year_delivery AS INT64) AS year_delivery,
    quarter,
    op_carrier,
    month,
    flyTo_country AS country,
    flight_type,
    SUM(revenue) AS revenue,
    SUM(rpk) AS rpk
  FROM
    `skytra-benchmark-rnd.matching.R7_flights`
  GROUP BY
    year_delivery,
    op_carrier,
    quarter,
    month,
    flyTo_country,
    flight_type),
  outbound_previous_year AS (
  SELECT
    CAST(year_delivery AS INT64) + 1 AS year_delivery_md,
    quarter,
    op_carrier,
    month,
    flyFrom_country AS country,
    flight_type,
    SUM(revenue) AS revenue,
    SUM(rpk) AS rpk
  FROM
    `skytra-benchmark-rnd.matching.R7_flights`
  WHERE
    dt_of_issue < "2019-05-14"
  GROUP BY
    year_delivery,
    op_carrier,
    quarter,
    month,
    flyFrom_country,
    flight_type),
  inbound_previous_year AS (
  SELECT
    CAST(year_delivery AS INT64) + 1 AS year_delivery_md,
    quarter,
    op_carrier,
    month,
    flyTo_country AS country,
    flight_type,
    SUM(revenue) AS revenue,
    SUM(rpk) AS rpk
  FROM
    `skytra-benchmark-rnd.matching.R7_flights`
  WHERE
    dt_of_issue < "2019-05-14"
  GROUP BY
    year_delivery,
    quarter,
    op_carrier,
    month,
    flyTo_country,
    flight_type),
  current_revenue AS (
  SELECT
    o.* EXCEPT(revenue),
    o.revenue AS outbound_current_year,
    IFNULL(i.revenue,
      0) AS inbound_current_year,
    o.rpk AS outboundrpk_current_year,
    IFNULL(i.rpk,
      0) AS inboundrpk_current_year
  FROM
    outbound_curent_year o
  INNER JOIN
    inbound_curent_year i
  ON
    o.country = i.country
    AND o.year_delivery = i.year_delivery
    AND o.flight_type = i.flight_type
    AND o.quarter = i.quarter
    AND o.month = i.month
    AND o.op_carrier = i.op_carrier
  WHERE
    o.year_delivery>2016 ),
  previous_revenue AS (
  SELECT
    o.* EXCEPT(revenue),
    o.revenue AS outbound_previous_year,
    IFNULL(i.revenue,
      0) AS inbound_previous_year,
    o.rpk AS outboundrpk_previous_year,
    IFNULL(i.rpk,
      0) AS inboundrpk_previous_year
  FROM
    outbound_previous_year o
  INNER JOIN
    inbound_previous_year i
  ON
    o.country = i.country
    AND o.year_delivery_md = i.year_delivery_md
    AND o.flight_type = i.flight_type
    AND o.quarter = i.quarter
    AND o.month = i.month
    AND o.op_carrier = i.op_carrier
  WHERE
    o.year_delivery_md>2016 )
SELECT
  c.*,
  IFNULL(outbound_previous_year,
    0) AS outbound_previous_year,
  IFNULL(inbound_previous_year,
    0) AS inbound_previous_year,
  outbound_current_year + inbound_current_year AS current_year_revenue,
  IFNULL(outbound_previous_year,
    0)+IFNULL(inbound_previous_year,
    0) AS previous_year_revenue,
  IFNULL(outboundrpk_previous_year,
    0) AS outboundrpk_previous_year,
  IFNULL(inboundrpk_previous_year,
    0) AS inboundrpk_previous_year,
  outboundrpk_current_year + inboundrpk_current_year AS current_year_rpk,
  IFNULL(outboundrpk_previous_year,
    0)+IFNULL(inboundrpk_previous_year,
    0) AS previous_year_rpk
FROM
  current_revenue c
LEFT JOIN
  previous_revenue p
ON
  c.country = p.country
  AND c.year_delivery = p.year_delivery_md
  AND c.flight_type = p.flight_type
  AND c.quarter = p.quarter
  AND c.op_carrier = p.op_carrier
  AND c.month = p.month
WHERE
  year_delivery > 2018

In [ ]:
df_0.head()

In [ ]:
df_0.to_gbq(destination_table='IN1092_new_products_dashboard.Country_per_month_domestic_international',
                               project_id='skytra-benchmark-rnd', if_exists = 'replace')